In [ ]:
# Attention, veuillez vous assurer de disposer du module tabulate avant l'exécution du code.
# Si vous ne l'avez pas, vous pouvez juste l'installer directement sur cmd via la commande : pip install tabulate 
from tabulate import tabulate 
from collections import deque
import os

In [5]:
class Automate:
    def __init__(self):
        self.alphabet = []
        self.etats = []
        self.initiaux = set()
        self.terminaux = set()
        self.transitions = {}


    def lire_fichier(self, nom_fichier):
        try:
            with open(nom_fichier, 'r') as fichier:
                lignes = fichier.readlines()
    
                if len(lignes) < 5:
                    print(f"Le fichier {nom_fichier} est mal formaté.")
                    return False
    
                nb_symboles = int(lignes[0].strip())
                self.alphabet = [chr(ord('a') + i) for i in range(nb_symboles)]
                
                nb_etats = int(lignes[1].strip())
                self.etats = list(range(nb_etats))
                
                ligne_initiaux = lignes[2].strip().split()
                nb_initiaux = int(ligne_initiaux[0])
                self.initiaux = set(map(int, ligne_initiaux[1:nb_initiaux + 1]))
                
                ligne_terminaux = lignes[3].strip().split()
                nb_terminaux = int(ligne_terminaux[0])
                self.terminaux = set(map(int, ligne_terminaux[1:nb_terminaux + 1]))
                
                nb_transitions = int(lignes[4].strip())
                if len(lignes) < 5 + nb_transitions:
                    print(f"Erreur : Nombre de transitions ({nb_transitions}) ne correspond pas au contenu du fichier.")
                    return False
                
                for i in range(5, 5 + nb_transitions):
                    try:
                        etat_depart, symbole, etat_arrivee = lignes[i].strip().split()
                        etat_depart = int(etat_depart)
                        etat_arrivee = int(etat_arrivee)
                        if etat_depart not in self.etats or etat_arrivee not in self.etats:
                            print(f"Erreur : État {etat_depart} ou {etat_arrivee} hors des états définis.")
                            return False
                        # Accepter "ε" comme symbole spécial
                        if symbole != "&" and symbole not in self.alphabet:
                            print(f"Erreur : Symbole {symbole} hors de l'alphabet.")
                            return False
                        if etat_depart not in self.transitions:
                            self.transitions[etat_depart] = {}
                        if symbole not in self.transitions[etat_depart]:
                            self.transitions[etat_depart][symbole] = set()
                        self.transitions[etat_depart][symbole].add(etat_arrivee)
                    except ValueError:
                        print(f"Erreur : Ligne {i+1} mal formatée : {lignes[i].strip()}")
                        return False
                
                print(f"Automate lu avec succès depuis {nom_fichier}.")
                return True
        except FileNotFoundError:
            print(f"Erreur : Le fichier {nom_fichier} n'existe pas.")
            return False
        except ValueError as e:
            print(f"Erreur de format dans {nom_fichier} : {e}")
            return False

    def fermeture_epsilon(self, etat, visite=None):
        """Calcule la fermeture ε d'un état (ensemble des états accessibles via ε-transitions).
    
        Args:
            etat: L'état de départ (entier ou frozenset).
            visite: Ensemble des états déjà visités pour éviter les cycles (par défaut None).
    
        Returns:
            set: Ensemble des états accessibles via des ε-transitions, incluant l'état de départ.
        """
        # Initialisation de l'ensemble visite pour éviter les boucles infinies
        if visite is None:
            visite = set()
        
        # La fermeture ε inclut toujours l'état de départ
        fermeture = {etat}
        visite.add(etat)
        
        # Vérifier si l'état a des ε-transitions
        if etat in self.transitions and "&" in self.transitions[etat]:
            # Explorer chaque état accessible via une ε-transition
            for etat_suivant in self.transitions[etat]["&"]:
                # Éviter de revisiter un état déjà exploré
                if etat_suivant not in visite:
                    # Ajouter récursivement les états accessibles depuis etat_suivant
                    fermeture.update(self.fermeture_epsilon(etat_suivant, visite))
        
        return fermeture


    def fermeture_epsilon_ensemble(self, etats):
        """Calcule la fermeture ε d'un ensemble d'états.
    
        Args:
            etats: Ensemble d'états (set) pour lesquels calculer la fermeture ε.
    
        Returns:
            set: Ensemble des états accessibles via des ε-transitions à partir des états donnés.
        """
        fermeture = set()
        # Calculer la fermeture ε pour chaque état et réunir les résultats
        for etat in etats:
            fermeture.update(self.fermeture_epsilon(etat))
        return fermeture



    def afficher_automate(self):
        """Affiche l'automate avec ses états initiaux, terminaux et une table de référence.
        La colonne ε est ajoutée uniquement s'il existe des ε-transitions."""
        def format_etat(etat):
            if isinstance(etat, frozenset):
                return ".".join(map(str, sorted(etat)))
            return str(etat)
    
        # Afficher les états initiaux et terminaux
        print(f"États initiaux : {', '.join(map(format_etat, self.initiaux))}")
        print(f"États terminaux : {', '.join(map(format_etat, self.terminaux))}\n")
    
        # Vérifier s'il existe des ε-transitions
        has_epsilon = any("&" in self.transitions[etat] for etat in self.transitions)
    
        # Construire la table de référence
        table = []
        for etat in self.etats:
            # Marqueurs pour les états initiaux (E) et terminaux (S)
            marqueur = ""
            if etat in self.initiaux:
                marqueur += "E"
            if etat in self.terminaux:
                marqueur += "S"
            ligne = [f"{marqueur}{format_etat(etat)}" if marqueur else format_etat(etat)]

            # Ajouter les transitions pour chaque symbole de l'alphabet
            for symbole in self.alphabet:
                if etat in self.transitions and symbole in self.transitions[etat]:
                    destinations = self.transitions[etat][symbole]
                    ligne.append(",".join(map(format_etat, sorted(destinations))))
                else:
                    ligne.append("--")
    
            # Ajouter les ε-transitions (uniquement si elles existent)
            if has_epsilon:
                if etat in self.transitions and "&" in self.transitions[etat]:
                    destinations = self.transitions[etat]["&"]
                    ligne.append(",".join(map(format_etat, sorted(destinations))))
                else:
                    ligne.append("--")
    
            table.append(ligne)
    
        # Définir les en-têtes de la table (inclure la colonne ε uniquement si nécessaire)
        headers = ["État"] + self.alphabet
        if has_epsilon:
            headers.append("&")
            print("Table de référence à ne pas confondre avec la :\n")
        
        print("Table de transitions :")
        print(tabulate(table, headers=headers, tablefmt="pretty"))

    
    def est_deterministe(self):
        """Vérifie si l'automate est déterministe."""
        if len(self.initiaux) != 1:
            print("L'automate n'est pas déterministe : il a plusieurs états initiaux.")
            return False
        
        for etat in self.transitions:
            for symbole in self.transitions[etat]:
                if len(self.transitions[etat][symbole]) > 1:
                    print(f"L'automate n'est pas déterministe : plusieurs transitions depuis l'état {etat} avec le symbole '{symbole}'.")
                    return False
        return True

    def est_complet(self):
        """Vérifie si l'automate est complet."""
        for etat in self.etats:
            if etat not in self.transitions:
                print(f"L'automate n'est pas complet : aucune transition depuis l'état {etat}.")
                return False
            for symbole in self.alphabet:
                if symbole not in self.transitions[etat] or not self.transitions[etat][symbole]:
                    print(f"L'automate n'est pas complet : il manque une transition depuis l'état {etat} avec le symbole '{symbole}'.")
                    return False
        return True

    def est_standard(self):
        """Vérifie si l'automate est standard."""
        if len(self.initiaux) != 1:
            print("L'automate n'est pas standard : il a plusieurs états initiaux.")
            return False
        
        etat_initial = next(iter(self.initiaux))
        for etat in self.transitions:
            for symbole in self.transitions[etat]:
                if etat_initial in self.transitions[etat][symbole]:
                    print(f"L'automate n'est pas standard : une transition pointe vers l'état initial {etat_initial} depuis l'état {etat} avec le symbole '{symbole}'.")
                    return False
        return True

    def standardiser(self):
        """Standardise l'automate si ce n'est pas déjà le cas."""
        if self.est_standard():
            print("L'automate est déjà standard.")
            return self

        anciens_initiaux = self.initiaux.copy()
        S0 = max(self.etats) + 1 if self.etats else 0
        self.etats.append(S0)
        self.initiaux = {S0}

        for ancien_initial in anciens_initiaux:
            if ancien_initial in self.transitions:
                for symbole, etats_arrivee in self.transitions[ancien_initial].items():
                    if S0 not in self.transitions:
                        self.transitions[S0] = {}
                    if symbole not in self.transitions[S0]:
                        self.transitions[S0][symbole] = set()
                    self.transitions[S0][symbole].update(etats_arrivee)

        if any(ancien_initial in self.terminaux for ancien_initial in anciens_initiaux):
            self.terminaux.add(S0)

        print("L'automate a été standardisé.")
        return self

    def determiniser(self):
        """Déterminise l'automate, en prenant en compte les ε-transitions.
    
        Returns:
            Automate: Un nouvel automate déterministe équivalent à l'automate initial.
        """
        # Vérifier si l'automate est déjà déterministe et sans ε-transitions
        if self.est_deterministe() and "&" not in [symbole for etat in self.transitions for symbole in self.transitions[etat]]:
            print("L'automate est déjà déterministe et sans ε-transitions.")
            return self
    
        # Créer un nouvel automate déterministe
        automate_det = Automate()
        automate_det.alphabet = self.alphabet.copy()
        
        # Calculer l'état initial de l'AFD : fermeture ε des états initiaux de l'AFN
        etat_initial_det = frozenset(self.fermeture_epsilon_ensemble(self.initiaux))
        automate_det.etats = [etat_initial_det]  # Liste des états de l'AFD
        automate_det.initiaux = {etat_initial_det}  # Un seul état initial
        # Si l'état initial contient un état terminal, il est terminal
        if any(etat in self.terminaux for etat in etat_initial_det):
            automate_det.terminaux.add(etat_initial_det)
        
        # File pour explorer les nouveaux états (BFS)
        file = deque([etat_initial_det])
        transitions_det = {}
    
        # Explorer tous les états de l'AFD
        while file:
            etat_courant = file.popleft()  # Prendre le prochain état à explorer
            transitions_det[etat_courant] = {}  # Initialiser ses transitions
            
            # Pour chaque symbole de l'alphabet
            for symbole in self.alphabet:
                etats_arrivee = set()
                # Pour chaque état de l'AFN dans l'état courant de l'AFD
                for etat in etat_courant:
                    # Vérifier les transitions avec ce symbole
                    if etat in self.transitions and symbole in self.transitions[etat]:
                        etats_arrivee.update(self.transitions[etat][symbole])
                
                # Calculer la fermeture ε des états atteints
                etats_arrivee = self.fermeture_epsilon_ensemble(etats_arrivee)
                
                # S'il y a des états atteints, créer un nouvel état dans l'AFD
                if etats_arrivee:
                    etat_arrivee_det = frozenset(etats_arrivee)
                    transitions_det[etat_courant][symbole] = {etat_arrivee_det}
                    
                    # Si cet état n'a pas encore été exploré, l'ajouter à la file
                    if etat_arrivee_det not in automate_det.etats:
                        automate_det.etats.append(etat_arrivee_det)
                        file.append(etat_arrivee_det)
                        # Vérifier si le nouvel état est terminal
                        if any(etat in self.terminaux for etat in etat_arrivee_det):
                            automate_det.terminaux.add(etat_arrivee_det)
        
        automate_det.transitions = transitions_det
        print("L'automate a été déterminisé (avec prise en compte des ε-transitions).")
        return automate_det

    

    def completer(self):
        """Complète l'automate si ce n'est pas déjà le cas, en gérant les états sous forme d'entiers ou de frozensets."""
        if self.est_complet():
            print("L'automate est déjà complet.")
            return self
    
        # Déterminer si les états sont des entiers ou des frozensets
        if self.etats and isinstance(self.etats[0], frozenset):
            # Après déterminisation : les états sont des frozensets
            # Trouver un entier unique pour créer un nouvel état puits
            # On extrait tous les entiers contenus dans les frozensets
            all_numbers = set()
            for etat in self.etats:
                all_numbers.update(etat)  # Ajoute tous les nombres du frozenset
            P = frozenset({max(all_numbers, default=-1) + 1})  # Nouvel état puits comme un frozenset singleton
        else:
            # Avant déterminisation : les états sont des entiers
            P = max(self.etats, default=-1) + 1 if self.etats else 0
    
        self.etats.append(P)
    
        # Compléter les transitions
        for etat in self.etats:
            if etat not in self.transitions:
                self.transitions[etat] = {}
            for symbole in self.alphabet:
                if symbole not in self.transitions[etat] or not self.transitions[etat][symbole]:
                    self.transitions[etat][symbole] = {P}
    
        print("L'automate a été complété.")
        return self

    def determiniser_et_completer(self):
        """Déterminise et complète l'automate."""
        if self.est_deterministe():
            print("L'automate est déterministe.")
            if self.est_complet():
                print("L'automate est déjà complet.")
                return self
            else:
                print("Complétion de l'automate déterministe.")
                return self.completer()
        else:
            print("Déterminisation et complétion de l'automate.")
            automate_det = self.determiniser()
            return automate_det.completer()

    def minimiser(self):
        """Minimise l'automate déterministe et complet."""
        AFDC = self.determiniser_et_completer()
        
        # Partition initiale
        terminaux = AFDC.terminaux
        non_terminaux = set(AFDC.etats) - terminaux
        partitions = [terminaux, non_terminaux] if non_terminaux else [terminaux]
        print("Partition initiale :")
        for i, p in enumerate(partitions):
            print(f"Groupe {i}: {', '.join(map(str, p))}")
        
        # Raffinement
        iteration = 1
        while True:
            nouvelles_partitions = []
            for groupe in partitions:
                if not groupe:
                    continue
                sous_groupes = {}
                for etat in groupe:
                    cle = tuple(
                        frozenset(AFDC.transitions.get(etat, {}).get(symbole, set()))
                        for symbole in AFDC.alphabet
                    )
                    if cle not in sous_groupes:
                        sous_groupes[cle] = set()
                    sous_groupes[cle].add(etat)
                nouvelles_partitions.extend(sous_groupes.values())
            
            if len(nouvelles_partitions) == len(partitions):
                break
            partitions = nouvelles_partitions
            print(f"\nPartition après raffinement {iteration}:")
            for i, p in enumerate(partitions):
                print(f"Groupe {i}: {', '.join(map(str, p))}")
            iteration += 1
        
        # Construction de l’automate minimal
        automate_min = Automate()
        automate_min.alphabet = AFDC.alphabet.copy()
        etat_groupe = {frozenset(groupe): i for i, groupe in enumerate(partitions) if groupe}
        automate_min.etats = list(range(len(etat_groupe)))
        automate_min.transitions = {i: {} for i in automate_min.etats}
        
        for groupe in partitions:
            if not groupe:
                continue
            etat_min = etat_groupe[frozenset(groupe)]
            if any(etat in AFDC.initiaux for etat in groupe):
                automate_min.initiaux.add(etat_min)
            if any(etat in AFDC.terminaux for etat in groupe):
                automate_min.terminaux.add(etat_min)
            
            etat_rep = next(iter(groupe))
            for symbole in AFDC.alphabet:
                if etat_rep in AFDC.transitions and symbole in AFDC.transitions[etat_rep]:
                    etats_arrivee = AFDC.transitions[etat_rep][symbole]
                    for g in partitions:
                        if etats_arrivee.issubset(g):
                            automate_min.transitions[etat_min][symbole] = {etat_groupe[frozenset(g)]}
                            break
        
        print("\nAutomate minimal construit.")
        print("Correspondance des états :")
        for i, groupe in enumerate(partitions):
            if groupe:
                print(f"État {i} : {', '.join(map(str, groupe))}")
        return automate_min

    def reconnaitre_mot(self, mot):
        """Vérifie si un mot est reconnu par un automate déterministe et complet.
    
        Args:
            mot (str): Le mot à tester (chaîne de caractères).
    
        Returns:
            bool: True si le mot est accepté, False sinon.
        """
        if not self.est_deterministe() or not self.est_complet():
            raise ValueError("L'automate doit être déterministe et complet pour reconnaître un mot.")
        
        # Commencer avec l'unique état initial
        etat_courant = next(iter(self.initiaux))
        
        # Parcourir chaque symbole du mot
        for symbole in mot:
            if etat_courant in self.transitions and symbole in self.transitions[etat_courant]:
                etat_courant = next(iter(self.transitions[etat_courant][symbole]))
            else:
                return False
        
        # Vérifier si l'état final est terminal
        return etat_courant in self.terminaux

    def tester_mots(self):
        """Permet à l'utilisateur de tester plusieurs mots."""
        while True:
            mot = input("Entrez un mot à tester (ou 'fin' pour quitter) : ")
            if mot == "fin":
                break
            if self.reconnaitre_mot(mot):
                print(f"Le mot '{mot}' est accepté.")
            else:
                print(f"Le mot '{mot}' est rejeté.")

    def automate_complementaire(self):
        """Crée l'automate reconnaissant le langage complémentaire."""
        if not self.est_deterministe() or not self.est_complet():
            print("L'automate doit être déterministe et complet pour créer le complémentaire.")
            return None
        
        automate_comp = Automate()
        automate_comp.alphabet = self.alphabet.copy()
        automate_comp.etats = self.etats.copy()
        automate_comp.initiaux = self.initiaux.copy()
        automate_comp.transitions = self.transitions.copy()
        automate_comp.terminaux = set(self.etats) - self.terminaux
        
        print("L'automate complémentaire a été construit.")
        return automate_comp

In [6]:
def main():
    while True:
        print("\n=== Menu Principal ===")
        print("1. Choisir un automate")
        print("2. Quitter")
        choix = input("Entrez votre choix : ")
        
        if choix == "2":
            break
        elif choix == "1":
            num_automate = input("Entrez le numéro de l'automate (ex. '8') : ")
            nom_fichier = f"automate_pool/exemple_{num_automate}.txt"
            if not os.path.exists(nom_fichier):
                print(f"Le fichier {nom_fichier} n'existe pas.")
                continue
            
            automate = Automate()
            if not automate.lire_fichier(nom_fichier):
                continue
            
            while True:
                print("\n=== Menu Automate ===")
                print("1. Afficher l'automate")
                print("2. Vérifier les propriétés")
                print("3. Standardiser l'automate")
                print("4. Déterminiser et compléter")
                print("5. Minimiser l'automate")
                print("6. Tester des mots")
                print("7. Créer l'automate complémentaire")
                print("8. Retour au menu principal")
                sous_choix = input("Entrez votre choix : ")
                
                if sous_choix == "8":
                    break
                elif sous_choix == "1":
                    automate.afficher_automate()
                elif sous_choix == "2":
                    print("Déterministe :", automate.est_deterministe())
                    print("Complet :", automate.est_complet())
                    print("Standard :", automate.est_standard())
                elif sous_choix == "3":
                    if not automate.est_standard():
                        automate = automate.standardiser()
                        automate.afficher_automate()
                    else:
                        print("L'automate est déjà standard.")
                elif sous_choix == "4":
                    automate = automate.determiniser_et_completer()
                    automate.afficher_automate()
                elif sous_choix == "5":
                    automate_min = automate.minimiser()
                    automate_min.afficher_automate()
                elif sous_choix == "6":
                    if not automate.est_deterministe() or not automate.est_complet():
                        print("L'automate doit être déterministe et complet pour tester des mots.")
                    else:
                        automate.tester_mots()
                elif sous_choix == "7":
                    if not automate.est_deterministe() or not automate.est_complet():
                        print("L'automate doit être déterministe et complet pour créer le complémentaire.")
                    else:
                        automate_comp = automate.automate_complementaire()
                        automate_comp.afficher_automate()
                else:
                    print("Choix invalide.")

In [7]:
if __name__ == "__main__":
    main()


=== Menu Principal ===
1. Choisir un automate
2. Quitter


Entrez votre choix :  1
Entrez le numéro de l'automate (ex. '8') :  1


Automate lu avec succès depuis automate_pool/exemple_1.txt.

=== Menu Automate ===
1. Afficher l'automate
2. Vérifier les propriétés
3. Standardiser l'automate
4. Déterminiser et compléter
5. Minimiser l'automate
6. Tester des mots
7. Créer l'automate complémentaire
8. Retour au menu principal


Entrez votre choix :  1


États initiaux : 0
États terminaux : 10

Table de référence à ne pas confondre avec la :

Table de transitions :
+------+----+----+-----+
| État | a  | b  |  &  |
+------+----+----+-----+
|  E0  | -- | -- | 1,4 |
|  1   | 2  | -- | 10  |
|  2   | -- | 3  | --  |
|  3   | -- | -- | 10  |
|  4   | -- | -- |  5  |
|  5   | 6  | -- |  6  |
|  6   | -- | 7  | --  |
|  7   | -- | -- |  8  |
|  8   | 9  | -- | 10  |
|  9   | -- | -- | 10  |
| S10  | -- | -- | --  |
+------+----+----+-----+

=== Menu Automate ===
1. Afficher l'automate
2. Vérifier les propriétés
3. Standardiser l'automate
4. Déterminiser et compléter
5. Minimiser l'automate
6. Tester des mots
7. Créer l'automate complémentaire
8. Retour au menu principal


...

Entrez votre choix :  8



=== Menu Principal ===
1. Choisir un automate
2. Quitter


Entrez votre choix :  8



=== Menu Principal ===
1. Choisir un automate
2. Quitter


Entrez votre choix :  2
